
<font size="20"><h3 align='center'>**_DSL Project_**</font><br>
    
    
<font size="5"><h3 align='center'><span style='color:Blue'> Predict impact of air quality on mortality rates(Kaggle Competition)</font>


<font size="3"><h3 align='center'>https://www.kaggle.com/c/predict-impact-of-air-quality-on-death-rates/overview</font><br>
    <br>


<font size="3">**Group Number - 2**<br>
<font size="3"> Mimansi Agarwal   &emsp;19BM6JP15</font>        
<font size="3"> Vaibhav HK Anand  &emsp;19BM6JP24</font>    
<font size="3"> Bankar Prasad Vilas &emsp;19BM6JP25</font> 


# Library

In [1]:

import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
import xgboost as xgb
from sklearn import metrics
import matplotlib.pyplot as plt
import seaborn as sns
from impyute.imputation.cs import mice
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import  make_scorer
import math

# Data import

In [44]:
train_full = pd.read_csv('full_data_imputation_train.csv', parse_dates=['date'], index_col='Id')
train = pd.read_csv('region_wise_imputed_train2.csv', parse_dates=['date'], index_col='Id')
test = pd.read_csv('test.csv', parse_dates=['date'], index_col='Id')
region=pd.read_csv('regions.csv')

In [45]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18403 entries, 1 to 16425
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   region          18403 non-null  object        
 1   date            18403 non-null  datetime64[ns]
 2   mortality_rate  18403 non-null  float64       
 3   O3              18403 non-null  float64       
 4   PM10            18403 non-null  float64       
 5   PM25            18403 non-null  float64       
 6   NO2             18403 non-null  float64       
 7   T2M             18403 non-null  float64       
 8   Code            18403 non-null  object        
 9   Region          18403 non-null  object        
dtypes: datetime64[ns](1), float64(6), object(3)
memory usage: 1.5+ MB


**_Missing values present in 4 columns, O3 ,PM10, PM25, NO2_**

In [46]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7886 entries, 18404 to 26289
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   region  7886 non-null   object        
 1   date    7886 non-null   datetime64[ns]
 2   O3      7886 non-null   float64       
 3   PM10    7886 non-null   float64       
 4   PM25    7886 non-null   float64       
 5   NO2     7886 non-null   float64       
 6   T2M     7886 non-null   float64       
dtypes: datetime64[ns](1), float64(5), object(1)
memory usage: 492.9+ KB


In [47]:
region

,Code,Region
0,E12000001,North East
1,E12000002,North West
2,E12000003,Yorkshire and The Humber
3,E12000004,East Midlands
4,E12000005,West Midlands
5,E12000006,East of England
6,E12000007,London
7,E12000008,South East
8,E12000009,South West


In [48]:
region.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Code    9 non-null      object
 1   Region  9 non-null      object
dtypes: object(2)
memory usage: 272.0+ bytes


# Exploratory data analysis

In [49]:
region.head()

,Code,Region
0,E12000001,North East
1,E12000002,North West
2,E12000003,Yorkshire and The Humber
3,E12000004,East Midlands
4,E12000005,West Midlands


#  Imputation

## Impution on full data

#  Feature Engeering

one hot encoding for regions

In [50]:
dummy1= pd.get_dummies(train['region'])
train_f = train.join(dummy1)

dummy2= pd.get_dummies(test['region'])
test_f = test.join(dummy2)

In [51]:
train_f.shape,test_f.shape

((18403, 19), (7886, 16))

doy and year

In [52]:
train_f['year'] = train_f['date'].map(lambda x: x.year)
test_f['year'] = test_f['date'].map(lambda x: x.year)

train_f['year^2'] = train_f['year']**2
test_f['year^2'] = test_f['year']**2

train_f['doy'] = train_f['date'].map(lambda x: x.dayofyear)
test_f['doy'] = test_f['date'].map(lambda x: x.dayofyear)

train_f['doy^2'] = train_f['doy']**2
test_f['doy^2'] = test_f['doy']**2

Log Transformation

In [53]:
train_f['NO2_log']=np.log(train_f['NO2'])
train_f['PM10_log']=np.log(train_f['PM10'])
train_f['PM25_log']=np.log(train_f['PM25'])

test_f['NO2_log']=np.log(test_f['NO2'])
test_f['PM10_log']=np.log(test_f['PM10'])
test_f['PM25_log']=np.log(test_f['PM25'])

C:\Users\miman\Anaconda3\lib\site-packages\pandas\core\series.py:679: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [54]:
train_f.info(),test_f.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18403 entries, 1 to 16425
Data columns (total 26 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   region          18403 non-null  object        
 1   date            18403 non-null  datetime64[ns]
 2   mortality_rate  18403 non-null  float64       
 3   O3              18403 non-null  float64       
 4   PM10            18403 non-null  float64       
 5   PM25            18403 non-null  float64       
 6   NO2             18403 non-null  float64       
 7   T2M             18403 non-null  float64       
 8   Code            18403 non-null  object        
 9   Region          18403 non-null  object        
 10  E12000001       18403 non-null  uint8         
 11  E12000002       18403 non-null  uint8         
 12  E12000003       18403 non-null  uint8         
 13  E12000004       18403 non-null  uint8         
 14  E12000005       18403 non-null  uint8         
 15  E1

(None, None)

In [55]:
total_f=total=pd.concat([train_f,test_f],sort=False)

In [56]:
total_f['T2M_rolling'] = total_f.groupby('region')['T2M'].transform(lambda x: x.rolling(30).mean())

In [57]:
total_f.shape

(26289, 27)

In [58]:
train_f=total_f[:18403].dropna()
test_f=total_f[18403:].drop(['mortality_rate'],axis=1)

In [59]:
train_f.shape,test_f.shape

((18025, 27), (7886, 26))

#  Modelling - Regression based models

In [60]:
X_train, X_test, y_train, y_test = train_test_split(train_f.drop(['mortality_rate'],axis=1), train_f['mortality_rate'], test_size=0.2)

In [61]:
X_train.columns

Index(['region', 'date', 'O3', 'PM10', 'PM25', 'NO2', 'T2M', 'Code', 'Region',
       'E12000001', 'E12000002', 'E12000003', 'E12000004', 'E12000005',
       'E12000006', 'E12000007', 'E12000008', 'E12000009', 'year', 'year^2',
       'doy', 'doy^2', 'NO2_log', 'PM10_log', 'PM25_log', 'T2M_rolling'],
      dtype='object')

In [62]:
RMSE= make_scorer(metrics.mean_squared_error)

In [63]:
predictors=['O3','PM10','PM25','NO2','T2M']

In [64]:
X=train_f.drop(['mortality_rate'],axis=1)
y=train_f['mortality_rate']

In [65]:
models = [
    LinearRegression(),
    linear_model.Lasso(alpha=0.1),
    Ridge(alpha=1.0),
    RandomForestRegressor(max_depth=2, random_state=0),
]
CV = 5
for model in models:
    model_name = model.__class__.__name__
    
    accuracies = cross_val_score(model,X[predictors], y,scoring=RMSE, cv=CV)
    
    print(model_name,"RMSE",np.sqrt(accuracies),"\n")
    print("average RMSE :", np.sqrt(accuracies).mean(),"\n","\n")

LinearRegression RMSE [0.31980505 0.22449029 0.21805517 0.36402558 0.23568978] 

average RMSE : 0.27241317521087177 
 

Lasso RMSE [0.32441552 0.22500054 0.21776906 0.36342753 0.23638645] 

average RMSE : 0.27339982014295733 
 

Ridge RMSE [0.31980514 0.22449028 0.21805514 0.36402552 0.23568976] 

average RMSE : 0.27241316789506526 
 

RandomForestRegressor RMSE [0.32482392 0.22368303 0.21741405 0.36731914 0.23723504] 

average RMSE : 0.2740950371114764 
 



In [66]:
predictors=['O3','PM10','PM25','NO2_log','T2M']

In [67]:
models = [
    LinearRegression(),
    linear_model.Lasso(alpha=0.1),
    Ridge(alpha=1.0),
    RandomForestRegressor(max_depth=2, random_state=0),
]
CV = 5
for model in models:
    model_name = model.__class__.__name__
    
    accuracies = cross_val_score(model,X[predictors], y,scoring=RMSE, cv=CV)
    
    print(model_name,"RMSE",np.sqrt(accuracies),"\n")
    print("average RMSE :", np.sqrt(accuracies).mean(),"\n","\n")

LinearRegression RMSE [0.31967942 0.22451915 0.21773052 0.363998   0.23531234] 

average RMSE : 0.2722478868338309 
 

Lasso RMSE [0.32441554 0.22500054 0.21776905 0.36342759 0.23638639] 

average RMSE : 0.27339981904351857 
 

Ridge RMSE [0.31967939 0.22451895 0.21773065 0.36399772 0.23531249] 

average RMSE : 0.27224783860164925 
 

RandomForestRegressor RMSE [0.32482392 0.22368303 0.21741405 0.36731914 0.23723504] 

average RMSE : 0.2740950371114764 
 



In [68]:
predictors=['O3','PM10','PM25_log','NO2_log','T2M']

In [69]:
models = [
    LinearRegression(),
    linear_model.Lasso(alpha=0.1),
    Ridge(alpha=1.0),
    RandomForestRegressor(max_depth=2, random_state=0),
]
CV = 5
for model in models:
    model_name = model.__class__.__name__
    
    accuracies = cross_val_score(model,X[predictors], y,scoring=RMSE, cv=CV)
    
    print(model_name,"RMSE",np.sqrt(accuracies),"\n")
    print("average RMSE :", np.sqrt(accuracies).mean(),"\n","\n")

LinearRegression RMSE [0.31959384 0.2244764  0.21769961 0.36364502 0.23511838] 

average RMSE : 0.2721066501442938 
 

Lasso RMSE [0.32441554 0.22500054 0.21776905 0.36342759 0.23638639] 

average RMSE : 0.27339981904351857 
 

Ridge RMSE [0.31959405 0.22447582 0.21770018 0.36364384 0.23511887] 

average RMSE : 0.27210655214402474 
 

RandomForestRegressor RMSE [0.32482392 0.22368303 0.21741405 0.36731914 0.23723504] 

average RMSE : 0.2740950371114764 
 



In [70]:
predictors=['O3','PM10_log','PM25_log','NO2_log','T2M']

In [71]:
models = [
    LinearRegression(),
    linear_model.Lasso(alpha=0.1),
    Ridge(alpha=1.0),
    RandomForestRegressor(max_depth=2, random_state=0),
]
CV = 5
for model in models:
    model_name = model.__class__.__name__
    
    accuracies = cross_val_score(model,X[predictors], y,scoring=RMSE, cv=CV)
    
    print(model_name,"RMSE",np.sqrt(accuracies),"\n")
    print("average RMSE :", np.sqrt(accuracies).mean(),"\n","\n")

LinearRegression RMSE [0.31950373 0.22442598 0.21752666 0.36442091 0.23505653] 

average RMSE : 0.27218676042264617 
 

Lasso RMSE [0.32441554 0.22500054 0.21776905 0.36342759 0.23638639] 

average RMSE : 0.27339981904351857 
 

Ridge RMSE [0.31950444 0.2244254  0.21752804 0.36441669 0.23505743] 

average RMSE : 0.27218640052450827 
 

RandomForestRegressor RMSE [0.32482392 0.22368303 0.21741405 0.36731914 0.23723504] 

average RMSE : 0.2740950371114764 
 



In [72]:
predictors=['O3','PM10_log','PM25_log','NO2_log','T2M_rolling']

In [73]:
models = [
    LinearRegression(),
    linear_model.Lasso(alpha=0.1),
    Ridge(alpha=1.0),
    RandomForestRegressor(max_depth=2, random_state=0),
]
CV = 5
for model in models:
    model_name = model.__class__.__name__
    
    accuracies = cross_val_score(model,X[predictors], y,scoring=RMSE, cv=CV)
    
    print(model_name,"RMSE",np.sqrt(accuracies),"\n")
    print("average RMSE :", np.sqrt(accuracies).mean(),"\n","\n")

LinearRegression RMSE [0.29990121 0.20445677 0.19911822 0.35334116 0.22223886] 

average RMSE : 0.2558112449777872 
 

Lasso RMSE [0.30779767 0.20678035 0.19980475 0.35380516 0.22272884] 

average RMSE : 0.2581833556616143 
 

Ridge RMSE [0.29990174 0.20445638 0.19911866 0.35333829 0.2222391 ] 

average RMSE : 0.25581083416874223 
 

RandomForestRegressor RMSE [0.30733398 0.2100518  0.20730363 0.35813483 0.23480915] 

average RMSE : 0.2635266767647039 
 



In [74]:
predictors=['O3','PM10_log','PM25_log','NO2_log','T2M_rolling','year','year^2','doy','doy^2']

In [75]:
models = [
    LinearRegression(),
    linear_model.Lasso(alpha=0.1),
    Ridge(alpha=1.0),
    RandomForestRegressor(max_depth=2, random_state=0),
]
CV = 5
for model in models:
    model_name = model.__class__.__name__
    
    accuracies = cross_val_score(model,X[predictors], y,scoring=RMSE, cv=CV)
    
    print(model_name,"RMSE",np.sqrt(accuracies),"\n")
    print("average RMSE :", np.sqrt(accuracies).mean(),"\n","\n")

LinearRegression RMSE [0.29146318 0.19604526 0.18443184 0.36173354 0.21151936] 

average RMSE : 0.24903863511430485 
 

Lasso RMSE [0.30831312 0.20208261 0.1796265  0.37443891 0.19972342] 

average RMSE : 0.2528369113358623 
 

Ridge RMSE [0.29152823 0.19739985 0.18339465 0.36234072 0.20990546] 

average RMSE : 0.24891378272045936 
 

RandomForestRegressor RMSE [0.31017182 0.20972111 0.20720556 0.36157547 0.23224796] 

average RMSE : 0.26418438208461265 
 



In [76]:
predictors=['T2M_rolling', 'year', 'year^2', 'doy', 'doy^2','E12000001', 'E12000002', 'E12000003', 'E12000004','E12000005', 'E12000006', 'E12000007', 'E12000008']

In [77]:
models = [
    LinearRegression(),
    linear_model.Lasso(alpha=0.1),
    Ridge(alpha=1.0),
    RandomForestRegressor(max_depth=2, random_state=0),
]
CV = 5
for model in models:
    model_name = model.__class__.__name__
    
    accuracies = cross_val_score(model,X[predictors], y,scoring=RMSE, cv=CV)
    
    print(model_name,"RMSE",np.sqrt(accuracies),"\n")
    print("average RMSE :", np.sqrt(accuracies).mean(),"\n","\n")

LinearRegression RMSE [0.23247568 0.18560885 0.20034362 0.16508042 0.19358304] 

average RMSE : 0.19541832128947606 
 

Lasso RMSE [0.30831321 0.20208264 0.1796265  0.37443876 0.19972339] 

average RMSE : 0.2528368997806702 
 

Ridge RMSE [0.23258775 0.18535738 0.20015901 0.16122874 0.19310904] 

average RMSE : 0.19448838274536168 
 

RandomForestRegressor RMSE [0.30000708 0.21395502 0.23363901 0.36157547 0.22935673] 

average RMSE : 0.26770666083646727 
 



In [78]:
predictors1=['NO2_log', 'year', 'year^2', 'doy', 'doy^2','PM10_log']
predictors2=['T2M_rolling', 'year', 'year^2', 'doy', 'doy^2','E12000001', 'E12000002', 'E12000003', 'E12000004','E12000005', 'E12000006', 'E12000007', 'E12000008']
predictors3=['E12000001', 'E12000002', 'E12000003', 'E12000004','E12000005', 'E12000006', 'E12000007', 'E12000008']

In [79]:
pred = [
    predictors1,
    predictors2,
    predictors3,
]
for preds in pred:
    #model_name = model.__class__.__name__
    
    accuracies = cross_val_score(LinearRegression(),X[preds], y,scoring=RMSE, cv=CV)
    
    print(model_name,"RMSE",np.sqrt(accuracies),"\n")
    print("average RMSE :", np.sqrt(accuracies).mean(),"\n","\n")

RandomForestRegressor RMSE [0.31533009 0.2083563  0.18571527 0.38014319 0.19939233] 

average RMSE : 0.25778743466294785 
 

RandomForestRegressor RMSE [0.23247568 0.18560885 0.20034362 0.16508042 0.19358304] 

average RMSE : 0.19541832128947606 
 

RandomForestRegressor RMSE [3.25087906e-01 2.76259647e-01 2.83855174e-01 2.68139960e-01
 3.61100784e+10] 

average RMSE : 7222015675.952733 
 



## Final Algo - mixed LR

In [80]:
a = LinearRegression()
b = LinearRegression()
c = LinearRegression()
ab = LinearRegression()

In [82]:

y0 = train_f['mortality_rate']



#modela
a_predictors = ['NO2_log', 'year', 'year^2', 'doy', 'doy^2','PM10_log']
a_X0 = train_f[a_predictors]
a.fit(a_X0, y0)



#modelb
b_predictors = [
        'T2M_rolling', 'year', 'year^2', 'doy', 'doy^2',
        'E12000001', 'E12000002', 'E12000003', 'E12000004',
        'E12000005', 'E12000006', 'E12000007', 'E12000008'
        ]
b_X0 = train_f[b_predictors]
b.fit(b_X0, y0)



#modelc
c_predictors = [
        'E12000001', 'E12000002', 'E12000003', 'E12000004',
        'E12000005', 'E12000006', 'E12000007', 'E12000008'
        ]
c_X0 = train_f[c_predictors]
c.fit(c_X0, y0)


region_effect_reduction = 0.85


# Model AB
# Produces the final solution. Combines A and "reduced B" by using their predictions as features for a linear regression.

# ab_X0 = pd.DataFrame({
#     'A': a.predict(a_X0),
#     'reduced B': b.predict(b_X0) - (c.predict(c_X0) * region_effect_reduction)
#     })
# 
ab_X0 = pd.DataFrame({
    'A': a.predict(a_X0),
    'reduced B': b.predict(b_X0) 
    })

ab.fit(ab_X0, y0)

# predict

a_X1 = test_f[a_predictors]
b_X1 = test_f[b_predictors]
#c_X1 = test_f[c_predictors]

#ab_X1 = pd.DataFrame({'A': a.predict(a_X1),'reduced B': b.predict(b_X1) - (c.predict(c_X1) * region_effect_reduction)})
ab_X1 = pd.DataFrame({'A': a.predict(a_X1),'reduced B': b.predict(b_X1)})

ab_y1 = ab.predict(ab_X1)



predictions = pd.DataFrame(index=test_f.index)
predictions['mortality_rate'] =  a.predict(a_X1)
predictions.to_csv('algo_final_train_full5.csv')


# Results
best LB socre for -(region_effect_reduction = 0.85)<br>
private score - 0.28948<br>
public score - 0.28784<br>


benchmark -<br>
private score - 0.29022<br>
public socre - 0.29259<br>